# 🐍Python Tricks - Black Magics

## Table of Content:
* [EAFP](#anchor1)
* [self assist](#anchor2)
* [eval](#anchor3)
* [str.find](#anchor4)
* [str.replace](#anchor5)
* [^ (xor)](#anchor6)
* [Sentinel](#anchor7)
* [+= ,](#anchor8)
* [List Comprehension with Break](#anchor9)
* [Integrated by Fractions](#anchor10)
* [Complex Number in Matrix](#anchor11)

***

## EAFP <a name="anchor1"></a>

use [**EAFP**](https://docs.python.org/2/glossary.html#term-eafp)(easier to ask for forgiveness than permission) against [**LBYL**](https://docs.python.org/2/glossary.html#term-lbyl) (look before you leap), actually this is the Pythonic design philosophy.

```python
"""LBYL"""
if "key" in dict_:
    value += dict_["key"]
    
"""EAFP"""
# downplays the importance of the key
try:
    value += dict_["key"]
except KeyError:
    pass
```

Let's write a Fibonacci example in a pythonic-style. We will provide the result in generator way and iterate in EAFP style.

In [1]:
"""pythonic fibonacci"""
#generator function
def fibonacci(n):
    a, b, counter = 0, 1, 0
    while counter <= n:
        yield a
        a, b = b, a + b
        counter += 1
        
f = fibonacci(5) #f is iterator object
while True:
    try:
        print (next(f), end=" ")
    except StopIteration:
        break

0 1 1 2 3 5 

#### Inspirations:
```python
"""used in detect cycle in linked list"""
# L141: determine there is a cycle in a linked list
def has_cycle_in_linked_list(head: ListNode) -> bool:
    try:
        slow = head
        fast = head.next
        while slow is not fast:
            slow = slow.next
            fast = fast.next.next
        return True
    except:
        return False

"""used in binary search and insert"""
# L334: given an unsorted array return whether an increasing subsequence (incontinuous) of length k exists or not in the array.
def increasing_triplet(nums: List[int], k: int) -> bool:
    try:
        inc = [float('inf')] * (k - 1)
        for x in nums:
            inc[bisect.bisect_left(inc, x)] = x
        return k == 0
    except:
        return True

"""used in eval"""
# L301: Remove the minimum number of invalid parentheses in order to make the input string valid. Return all possible results.
def remove_invalid_parentheses(s: str) -> List[str]:
    def isvalid(s):
        try:
            eval('0,' + ''.join(filter('()'.count, s)).replace(')', '),'))
            return True
        except:
            pass
    level = {s}
    while True:
        valid = list(filter(isvalid, level))
        if valid:
            return valid
        level = {s[:i] + s[i+1:] for s in level for i in range(len(s))}
```

## self assist <a name="anchor2"></a>

a trick of dynamic property, but a bit intrusive

#### Inspiration:
```python
"""self as dummy node in linked list"""
def traverse_linked_list(self, head: TreeNode) -> TreeNode:
    pre, pre.next = self, head
    while pre.next:
        self.process_logic(pre.next)
    return self.next  # return head node
```

## eval <a name="anchor3"></a>
eval() executes arbitrary strings as Python code. But note that eval may become a potential security risk.

#### Inspiration:
```python
"""build a tree constructor and use eval to execute"""
# L536: construct binary tree from string
def str2tree(s: str) -> TreeNode:
    def t(val, left=None, right=None):
        node, node.left, node.right = TreeNode(val), left, right
        return node
    return eval('t(' + s.replace('(', ',t(') + ')') if s else None
```

## str.find <a name="anchor4"></a>
This is clever use of str.find. Although this usage is not very generalizable, the idea can be used for reference.

#### Inspiration:
```python
"""use find to construct a mechanism: if val is # return 1, if not return -1"""
# L331: verify preorder serialization of a binary tree, if it is a null node, we record using a sentinel value #
def is_valid_serialization(preorder: str) -> bool:
    need = 1
    for val in preorder.split(','):
        if not need:
            return False
        need -= ' #'.find(val)
    return not need
```

## str.replace <a name="anchor5"></a>

Use str.replace to implement a string version's union-find (disjoint-set).

#### Inspiration:
```python
"""convert int to unicode char, and use str.replace to merge the connected nodes by reduce"""
# L323: given n nodes from 0 to n - 1 and a list of undirected edges, find the number of connected components in an undirected graph.
def count_connected_components(n: int, edges: List[List[int]]) -> int:
    return len(set(reduce(lambda s, edge: s.replace(s[edge[1]], s[edge[0]]), edges, ''.join(map(chr, range(n))))))
```

## ^ (xor) <a name="anchor6"></a>
^ is usually used for removing even exactly same numbers and save the odd, or save the distinct bits and remove the same.

#### Inspirations:
```python
"""Use ^ to remove even exactly same numbers and save the odd, or save the distinct bits and remove the same."""
"""bit manipulate: a^b^b = a"""
# L268: Given an array containing n distinct numbers taken from 0, 1, 2, ..., n, find the one that is missing from the array.
def missing_number(nums: List[int]) -> int:
    res = 0
    for i, e in enumerate(nums):
        res = res ^ i ^ e
    return res ^ len(nums)

"""simply find the first index whose "partner index" (the index xor 1) holds a different value."""
# L540: find the single element, in a sorted array where every element appears twice except for one. 
def single_non_duplicate(nums: List[int]) -> int:
    lo, hi = 0, len(nums) - 1
    while lo < hi:
        mid = (lo + hi) // 2
        if nums[mid] == nums[mid ^ 1]:
            lo = mid + 1
        else:
            hi = mid
    return nums[lo]

"""parity in triple comparisons"""
# L81: search a target in a rotated sorted array
def search_in_rotated_sorted_arr(nums: List[int], target: int) -> int:
    """I have three checks (nums[0] <= target), (target <= nums[i]) and (nums[i] < nums[0]), and I want to know
    whether exactly two of them are true. They can't all be true or all be false (check it), so I just need to
    distinguish between "two true" and "one true". Parity is enough for that, so instead of adding them I xor them"""
    self.__getitem__ = lambda i: (nums[0] <= target) ^ (nums[0] > nums[i]) ^ (target > nums[i])
    i = bisect.bisect_left(self, True, 0, len(nums))
    return i if target in nums[i:i+1] else -1
```

## Sentinel <a name="anchor7"></a>

Sentinel can make the program simpler: a mechanism to distinguish useful data from placeholders which indicate data is absent.

For example, we can put the key we search after the end of the array, this ensures that we will eventually find the element. When we find it, we only have to check whether we found a real element or just the sentinel.

Here we compare two similar built-in functions: str.index without sentinel and str.find with sentinel.

#### Sentinel in Python:
```python
"""find vs index"""    
"""index without sentinel"""
try:
    i = a.index(b)
except:
    return

"""index with sentinel"""
i = a.find(b)
if i == -1:
    return

"""sentinel in dict.get"""
sentinel = object()
value = my_dict.get(key, sentinel)
if value is not sentinel:
    # Do something with value

"""sentinel in iter"""
blocks = ''.join(iter(partial(f.read, 32), ''))
```

#### Inspirations:
```python        
"""add a sentinel n at the end (which is the appropriate last insertion index then)"""
# L47: given a collection of numbers that might contain duplicates, return all possible unique permutations.
def permute_unique(nums: List[int]) -> List[List[int]]:
    perms = [[]]
    for n in nums:
        perms = [p[:i] + [n] + p[i:]
                 for p in perms
                 for i in range((p + [n]).index(n) + 1)]
    return perms

"""sentinel in matrix"""    
def traverse_neighbors(matrix: List[List[int]]):
    m, n = len(matrix), len(matrix[0])
    """augment matrix to void length check by sentinel"""
    matrix += [0] * n,
    for row in matrix:
        row.append(0)
    
    for i in range(m):
        for j in range(n):
            # construct neighbor iterator
            for I, J in (i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1):
                """no need to check boundary"""
                process_neighbor_logic(matrix[I][J])

"""functional sentinel"""
def get_element(matrix: List[List[int]], i: int, j: int) -> int:
    return matrix[i][j] if 0 <= i < m and 0 <= j < n else -1
```

## += , <a name="anchor8"></a>

less to type and more clear, even faster according to the test. but if not familiar with, may bring confusion.

In [2]:
""", means convert to tuple, += element, equals to append(element)"""
arr = [1, 2, 3]
arr += 4,
arr

[1, 2, 3, 4]

## List Comprehension with Break <a name="anchor9"></a>

We know we cannot use branch logic (conditional execution) like if/else in list comprehension, so how to simulate break in list comprehension?

Notice it is just a study and exploration of list comprehension. Since this trick is too hackish with poor readability and performance, so it should not be used in production code. But the techniques and ideas can be used for reference and may provide some inspirations.

Let's take this Stack Overflow question as an example:
>How can I stop the iteration of list comprehension when a particular element is found.

```python
# pseudo code
new_list=[a for a in origin_list break if a==break_elment]
```

Here is the hackish solution:

In [3]:
# https://stackoverflow.com/a/56054962/11263560
origin_list = [1, 2, 3, 3, 4, 3, 5]
break_elment = 3

new_list = [a for end in [[]] for a in origin_list
         if not end and not (a == break_elment and end.append(42))]
new_list

[1, 2, 3]

There are many techniques used in this trick:

1. The key point is building an `end` condition in list comprehension. Skip the rest elements when end is not empty (Actually not break out, but indeed in logic).
2. How to initialize a variable (`end`) in a list comprehension?  Here is a trick to wrap it in a for list: `for end in [[]]`.
3. How to implement branch logic in a list comprehension? Use lazy evaluation technique in `and`/`or` to divide branch logics.


#### Inspiration:
```python
# https://stackoverflow.com/a/55671533/11263560
# How can I break a list comprehension based on a condition, for instance when the number 412 is found?
# requirement in pseudo code
even = [n for n in numbers if 0 == n % 2 and break if n == 412]

"""use end condition"""
even = [n for end in [[]] for n in numbers
        if (False if end or n != 412 else end.append(42))
        or not end and not n % 2]

# https://stackoverflow.com/q/55646039/11263560
"""use push & pop to record last pair"""
res = [last.pop() and last.append(b) or b for last in [[desired_list[0]]] for a, b in 
       zip([desired_list[0]] + desired_list, desired_list) if abs(a[1] - b[1]) <= 5 and a == last[0]]

"""use end condition"""
res = [b for end in [[]] for a, b in zip([desired_list[0]] + desired_list, desired_list) 
       if (False if end or abs(a[1] - b[1]) <= 5 else end.append(42)) or not end and abs(a[1] - b[1]) <= 5]
```

## Integrated by Fractions  <a name="anchor10"></a>

Integrate several dimensions into one dictionary by the index with fractions.

#### Inspiration:
```python
# L562: given a 01 matrix, find the longest line of consecutive one. the line could be horizontal, vertical, diagonal or anti-diagonal.
def longest_line(matrix: List[List[int]]) -> int:
    max_len = 0
    cur_len = defaultdict(int)
    for i, row in enumerate(matrix):
        for j, v in enumerate(row):
            """merge row, col, analog, anti-analog into one dict by fractions"""
            for key in i, j + .1, i + j + .2, i - j + .3:    # analog: i+j, anti-analog: i-j
                cur_len[key] = (cur_len[key] + 1) * v   # accumulate util v turn to zero
                max_len = max(max_len, cur_len[key])
    return max_len
```

## Complex Number in Matrix <a name="anchor11"></a>

use complex number in 2d representation, and visit 4-directions by imaginary unit calculation.

#### Inspirations:
```python
"""simplify two-dimension index into one-dimension by complex number"""
# traverse neighbors in matrix
def traverse_neighbor_by_complex(matrix: List[List[int]]) -> None:
    matrix = {i + 1j * j: v for i, row in enumerate(matrix) for j, v in enumerate(row)}
    for z in matrix:
        """visit 4-directional neighbor by complex calculation"""
        for k in range(4):
            process_neighbor_logic(matrix.get(z + 1j ** k))
       
# L657: given a sequence of robot 4-directional moves "LRUD", judge if this robot ends up at (0, 0) after it completes its moves.
def judge_circle(moves: str) -> bool:
    """D: 1j**-1=-1j, R: 1j**0=1+0j, U: 1j**1=1j, L: 1j**2=-1+0j, result in D+U=0 and L+R=0"""
    return not sum(1j**'RUL'.find(m) for m in moves)

"""use complex number to represent island(simplify 2d -> 1d and turn into a sparse representation)"""
# L695: an island is a group of 1's connected 4-directionally, find the maximum area of an island in the given 2D array.
def max_area_of_island(grid: List[List[int]]) -> int:
    grid = {i + j * 1j: val for i, row in enumerate(grid) for j, val in enumerate(row)}
    """calculate the area of paricular island by visit neigher complex calculat"""
    def area(z):
        return grid.pop(z, 0) and 1 + sum(area(z + 1j ** k) for k in range(4))
    return max(map(area, set(grid)))
```

***